
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [63]:
import os
Root = "/content"
os.chdir(Root)

In [64]:
ls

data/  data.zip  sample_data/


In [65]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [66]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [67]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'panic',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'panic', 'disgust']

In [71]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [81]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [82]:
x_train

array([[-5.28475525e+02,  4.68571091e+01, -3.67016387e+00, ...,
         6.54241710e-04,  3.29136848e-04,  9.66246735e-05],
       [-4.61271820e+02,  2.29928493e+01, -3.58219147e+01, ...,
         3.00771353e-04,  1.76716712e-04,  1.15658870e-04],
       [-4.80286407e+02,  2.78660526e+01, -1.06364164e+01, ...,
         1.85660459e-03,  1.39153481e-03,  8.21309222e-04],
       ...,
       [-6.59763672e+02,  4.14557266e+01,  7.44626141e+00, ...,
         2.22209244e-04,  1.17061478e-04,  1.66183949e-04],
       [-5.44744385e+02,  7.57021484e+01, -7.10741949e+00, ...,
         2.04873431e-05,  1.28833863e-05,  1.45689773e-05],
       [-6.71299377e+02,  3.41379204e+01, -6.10329688e-01, ...,
         1.14579489e-04,  9.84936705e-05,  5.24207717e-05]])

In [83]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [84]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [85]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [86]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [87]:
#Predict for the test set
y_pred=model.predict(x_test)

In [88]:
y_pred

array(['panic', 'happy', 'happy', 'happy', 'panic', 'calm', 'happy',
       'disgust', 'happy', 'disgust', 'panic', 'calm', 'panic', 'disgust',
       'happy', 'happy', 'calm', 'happy', 'happy', 'calm', 'calm', 'calm',
       'calm', 'panic', 'calm', 'panic', 'calm', 'calm', 'calm', 'happy',
       'happy', 'disgust', 'happy', 'panic', 'disgust', 'disgust', 'calm',
       'panic', 'happy', 'panic', 'disgust', 'panic', 'happy', 'happy',
       'panic', 'panic', 'panic', 'panic', 'panic', 'calm', 'disgust',
       'calm', 'panic', 'happy', 'disgust', 'disgust', 'calm', 'disgust',
       'disgust', 'happy', 'happy', 'calm', 'calm', 'happy', 'panic',
       'calm', 'happy', 'panic', 'calm', 'panic', 'calm', 'disgust',
       'calm', 'happy', 'calm', 'calm', 'calm', 'panic', 'panic', 'calm',
       'happy', 'disgust', 'happy', 'panic', 'happy', 'panic', 'panic',
       'calm', 'panic', 'happy', 'calm', 'calm', 'calm', 'disgust',
       'calm', 'panic', 'calm', 'panic', 'panic', 'calm', 'dis

In [89]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 72.92%


In [90]:
from sklearn.metrics import accuracy_score, f1_score

In [91]:
f1_score(y_test, y_pred,average=None)

array([0.80769231, 0.72527473, 0.66666667, 0.70707071])

In [92]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

Actual Predicted
0     panic     panic
1     happy     happy
2     panic     happy
3     happy     happy
4     panic     panic
5      calm      calm
6     panic     happy
7   disgust   disgust
8     panic     happy
9   disgust   disgust
10    panic     panic
11  disgust      calm
12  disgust     panic
13  disgust   disgust
14    happy     happy
15  disgust     happy
16     calm      calm
17  disgust     happy
18    happy     happy
19     calm      calm

In [93]:
import pickle
# Writing different model files to file
with open( 'trained_model.sav', 'wb') as f:
    pickle.dump(model,f)

In [106]:
filename = 'trained_model.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/data/Actor_01/03-01-04-02-02-02-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['panic'], dtype='<U7')

In [ ]:
feature